<a href="https://colab.research.google.com/gist/mohzeki222/7faebc6a45a511fa33c769ed88ca740b/chapter1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

オリジナルはこちらです。
https://altema.is.tohoku.ac.jp/QA4U/

### **実感の湧く問題を解いてみよう**

少し具体的に意味のある問題を解いてみることにしましょう。


**いくつかの荷物があり、それぞれには重さが異なるものとします。
それを運ぶ2人がいて、重さが均等になるようにその荷物を2つのグループに分けたい。**

どのようにしたら良いでしょうか？
ここで考えなければならないのは**QUBO行列を作る**ことです。
ただしいきなり$Q_{ij}$のことを思っても何も思い浮かばないものです。
そこで重要となるのが**数式によるモデリング**です。
ここが量子アニーリングの研究開発を行う上での成長ポイントです。
量子そのものの前に、数理モデリングに挑戦する必要があるのです。

重さを持ついくつかの荷物があるというのだから、その重さを$w_i$としましょう。
$N$個あるとして、合計した重さは$W=\sum_{i=1}^N w_i$です。

（こうやって何も与えられていないところで**自分で文字式を立てる**ところから訓練です）


2人のうちAさんがその荷物を取る場合$x_i=1$として、取らない場合は$x_i=0$とすると、
Aさんが持つ荷物の重さの合計は、
\begin{equation}
W_A = \sum_{i=1}^N w_i x_i
\end{equation}
逆にBさんは残りの荷物を持つので
\begin{equation}
W_B = W - W_A
\end{equation}
となります。
これらが等しくなるというのだから、
\begin{equation}
W_A - W_B
\end{equation}
という引き算をしたズレが$0$になれば完璧です。
もしくは非常に小さいものとなってくれれば嬉しい。
ただ$W_A$がわずかに大きくても仕方ないし、小さくても仕方ない。正負の値どちらでも良いからとにかく**ズレの大きさ**ができるだけ小さいことが望ましいというわけです。
できるだけ小さい、すなわちズレの大きさが最小になるような組み合わせを見つければ良いですよね。
そうするとコスト関数として、次のようなものを考えてみましょう。
\begin{equation}
E({\bf x}) = \left( W_A - W_B \right)^2 = \left( 2W_A - W \right)^2
\end{equation}
ここに$W_A$の具体的な形として先ほど準備しておいた形を入れてみましょう。
\begin{equation}
E({\bf x}) = \left( 2\sum_{i=1}^N w_i x_i - W \right)^2
\end{equation}



何か近い形になってきましたね。二乗をするというのは同じものを掛け算するという意味です。
シグマ記号は嫌らしいけれども意味はとにかく足し算をするというものでした。
下にある$i=1$は$i$という文字を$1$から動かして上にある$N$まで変えて足し算してくださいね、ということです。
だったら$i$という文字は仮置きをしているだけですから、別の文字を使っても良いですね。


\begin{equation}
E({\bf x}) = \left( 2\sum_{i=1}^N w_i x_i - W \right)\left( 2\sum_{j=1}^N w_j x_j - W \right)
\end{equation}

この掛け算を展開してみましょう。

\begin{equation}
E({\bf x}) = 4\sum_{i=1}^N\sum_{j=1}^N w_iw_j x_ix_j - 2W\sum_{i=1}^N w_i x_i  - 2W\sum_{j=1}^N w_j x_j + W^2
\end{equation}

ここで第二項と第三項で同じ和が2つ出ていますので、まとめておきましょう。
\begin{equation}
E({\bf x}) = 4\sum_{i=1}^N\sum_{j=1}^N w_iw_j x_ix_j - 4W\sum_{i=1}^N w_i x_i + W^2
\end{equation}


$x_ix_j$という部分が出てきましたね。その係数は$4w_iw_j$で、この部分がQUBO行列に相当します。$x_i$が$0$と$1$のどちらかの値を持つことから、$x_i$が$x_ix_i$と同じことを思いつくと、第二項は、$-4W\sum_{i=1}^N w_i x_ix_i$という意味で、QUBO行列のうち添え字が同じ$i$と$i$のとき、対角成分のことを示していることがわかります。
これらの事実からQUBO行列を作るプログラムの発想ができます。

In [49]:
import numpy as np

In [50]:
N = 10
w = np.random.rand(N)

まず例えば$N=10$個の荷物について、その重さを適当な乱数で決めることにしましょう。

np.random.rand()で$0$から$1$の範囲にある適当な数値が出ます。

この係数からQUBO行列を作ります。
まず最初に全重量の計算です。

In [51]:
W = np.sum(w)

numpyのsum()を利用すれば全重量の合計が計算できます。

In [52]:
Q = np.zeros(N**2).reshape(N,N)

まずはQUBO行列を格納する場所を用意しましょう。np.zeros()はその名の通り、全成分を0で埋めたものを並べて作ります。これを.reshape(N,N)でN×Nの行列に整形します。

In [53]:
for i in range(N):
  for j in range(N):
    Q[i][j] = 4*w[i]*w[j]

まず第一項の計算をここで実行しています。for文を使って、iとjを動かしながら、$Q_{ij}$すなわちQ[i][j]に値を入れていきます。その値は$4w_iw_j$ですから、その結果を入れていきます。
次に第二項をQUBO行列の対角成分に追加しましょう。

In [54]:
for i in range(N):
  Q[i][i] = Q[i][i] - 4*W*w[i]

ここで注意して欲しいのが、第一項で計算した結果に追加するので、元からあるQ[i][i]に引き算をするようにしています。

これでQUBO行列の作成完了です。

### **QUBO行列の設定方法について**

QUBO行列はだんだんと巨大なものになってくると、そのデータ転送にも時間がかかるようになり、せっかくの量子アニーリングマシンのパワーを台無しにしてしまうことがあります。
データの転送量を抑えるためにも、不要な成分についてはその情報を送らないようにしておくと良いでしょう。
python上ではnumpyによるarray形式で行列を扱うことが多いのですが、代わりにdict形式でデータを送りましょう。


In [55]:
Qdict = {}
for i in range(N):
  for j in range(N):
    if Q[i][j] != 0.0:
      Qdict[(i,j)] = Q[i][j]

dict形式の初期化は{}で中身は空欄にしておくだけです。
Qdict[(i,j)]のように(i,j)でどの成分に値があるのかを指定して、その値を代入します。
ここではif文を使い、ゼロではないところだけ、Qdictのデータとして格納しています。
出来上がったものを確認したい場合にはQdictとそのまま打って実行したり、print(Qdict)と実行してみましょう。

In [56]:
print(Qdict)

{(0, 0): -9.496160965427809, (0, 1): 1.335779814328449, (0, 2): 0.30117057142631193, (0, 3): 0.9135904223540371, (0, 4): 2.3042439593904915, (0, 5): 1.393727774808067, (0, 6): 1.4087426283641773, (0, 7): 0.19672826173765257, (0, 8): 1.476977938008617, (0, 9): 0.16519959501000425, (1, 0): 1.335779814328449, (1, 1): -8.079885194489307, (1, 2): 0.24890088262956672, (1, 3): 0.7550321447707428, (1, 4): 1.9043306674021012, (1, 5): 1.1518392107575048, (1, 6): 1.164248159895432, (1, 7): 0.16258506849718124, (1, 8): 1.220640883515702, (1, 9): 0.13652836269262628, (2, 0): 0.30117057142631193, (2, 1): 0.24890088262956672, (2, 2): -2.0145077549111168, (2, 3): 0.1702327434856159, (2, 4): 0.42935845349218493, (2, 5): 0.25969854430647366, (2, 6): 0.2624963185074174, (2, 7): 0.036657117782019535, (2, 8): 0.2752108606908096, (2, 9): 0.030782262590716637, (3, 0): 0.9135904223540371, (3, 1): 0.7550321447707428, (3, 2): 0.1702327434856159, (3, 3): -5.764776654750435, (3, 4): 1.3024438908805405, (3, 5): 0.

どこの成分に重要な非零の行列成分があるのかを指定する形になっています。
上記の問題では非零成分のない問題になっていますので影響はさほどありませんが、
基本的なテクニックとして知っておくと良いでしょう。

（正直この入力の違いだけでハイブリッドソルバーなどでは如実に性能が変わります）

### **シミュレータを活用しよう**

さて上記のように用意した量子アニーリングマシンは、利用回数には制限があり、大事に使いたいところでしょう。演習の際には豊富なマシンタイムを利用することのできるAPI tokenを発行する予定ですが、講義の間の試し利用の場合には、代わりになるシミュレータを利用すると良いでしょう。
その一つが**株式会社Jijの開発するOpenJij**です。

OpenJijは量子アニーリングマシンのシミュレータを搭載するオープンソースソフトウェアです。


基本的な利用方法は、これまでと同じようにQUBO行列を作ったのちにsamplerに投入するだけです。その際にOpenJijのsamplerを利用します。

その前にまずはOpenJijのインストールが必要です。

In [57]:
pip install openjij

再びpip installを利用して、OpenJijをインストールします。
その後にimport SQASamplerを実行してsamplerの準備を行いましょう。

In [58]:
from openjij import SQASampler
sampler = SQASampler()

準備はこれだけです。SQAというのはシミュレーテッド量子アニーリングというもので、
**量子モンテカルロ法**という計算技術を活用して、**量子アニーリングのシミュレーション**を行っています。
オプションでそのシミュレーションのパラメータ等を設定することができますが、とりあえず前に進めていきましょう。

In [59]:
sampleset = sampler.sample_qubo(Qdict, num_reads=10)

In [60]:
print(sampleset.record)

[([1, 0, 1, 1, 0, 1, 0, 1, 1, 0], -19.02778174, 1)
 ([1, 1, 1, 1, 0, 0, 1, 1, 0, 0], -19.07137116, 1)
 ([1, 0, 1, 1, 0, 1, 1, 1, 0, 1], -18.98089889, 1)
 ([1, 0, 1, 1, 0, 1, 1, 0, 0, 0], -19.09656156, 1)
 ([0, 0, 1, 1, 0, 1, 1, 0, 1, 0], -19.09787861, 1)
 ([0, 0, 1, 0, 1, 1, 0, 0, 1, 0], -19.09628796, 1)
 ([1, 0, 1, 1, 1, 0, 0, 1, 0, 1], -19.09810471, 1)
 ([0, 1, 1, 0, 0, 1, 1, 1, 1, 1], -18.77765324, 1)
 ([0, 0, 1, 1, 0, 1, 1, 1, 1, 0], -19.08903115, 1)
 ([0, 1, 0, 1, 1, 0, 1, 0, 0, 1], -18.89830563, 1)]


OpenJijではdict形式で問題を受け付けますので、注意してください。

（先程の手順でnumpy array形式だったものをdict形式に直しておきましょう）

結構いい答えが出てきたのではないでしょうか。これでとりあえずは十分ですよね。
ただ問題のサイズが大きくなるにつれて結果が次第に悪化してくることがありますのでご注意を。
それはパラメータの設定で改善する可能性があります。